# ResNet-50 Transfer Learning Testing
In this notebook, we'll be loading the last trained checkpoint and comparing it to ResNet-50 trained on ImageNet, using Flower Classification CC BY 4.0 dataset, [available here](https://data.mendeley.com/datasets/738sdjm6h9/1).

We try to use the same hyperparameters as [this notebook](https://github.com/ovh/ai-training-examples/blob/main/notebooks/computer-vision/image-classification/tensorflow/resnet50/notebook-resnet-transfer-learning-image-classification.ipynb), so that we're able to compare it to a ResNet-50 trained on ImageNet.


### Constants and imports

In [1]:
import os
import torch
import requests
import tarfile
from tqdm.notebook import tqdm
from lightning.pytorch import Trainer, seed_everything, LightningModule
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import utils
from torchvision.io import read_image
from torchvision.transforms import v2

CHECKPOINTS_DIRECTORY = 'checkpoints'
FLOWER_CHECKPOINTS_DIRECTORY = 'flowers-checkpoints'
FLOWER_DATASET_DIRECTORY = 'flowers'
FLOWER_DATASET_ARCHIVE_FILENAME = 'flower_photos.tgz'

# Notebook and Keras Adam default values
USE_RANDOM_MODEL = False

EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-7

TRAIN_DATASET_RATIO = 0.7
VAL_DATASET_RATIO = 0.2
TEST_DATASET_RATIO = 0.1

FLOWER_CLASSIFICATION_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

### Download Flower Classification dataset if it isn't downloaded

In [2]:
if not os.path.exists(FLOWER_DATASET_DIRECTORY):
    os.makedirs(FLOWER_DATASET_DIRECTORY)

In [3]:
is_downloaded = False
is_unpacked = False

def download_file(url, path):
    image_data = requests.get(url, stream=True)
    if image_data.status_code == 200:
        with open(path, 'wb') as f:
            for chunk in image_data.iter_content(2048):
                f.write(chunk)

def extract_flower_archive(archive_path, target_path):
    tar = tarfile.open(archive_path, 'r')
    for file in tar.getmembers():
        if file.isdir() or '.jpg' in file.name:
            tar.extract(file, target_path)

flower_directory_listing = os.listdir(FLOWER_DATASET_DIRECTORY)
if len(flower_directory_listing) != 0:
    if 'flower_photos.tgz' in flower_directory_listing:
        is_downloaded = True
    if 'flower_photos' in flower_directory_listing:
        flower_photos_listing = os.listdir(os.path.join(FLOWER_DATASET_DIRECTORY, 'flower_photos'))
        if 'daisy' in flower_photos_listing and 'roses' in flower_photos_listing and 'dandelion' in flower_photos_listing and 'sunflowers' in flower_photos_listing and 'tulips' in flower_photos_listing:
            in_unpacked = True

if not is_unpacked:
    if not is_downloaded:
        download_file(FLOWER_CLASSIFICATION_URL, os.path.join(FLOWER_DATASET_DIRECTORY, FLOWER_DATASET_ARCHIVE_FILENAME))
    extract_flower_archive(os.path.join(FLOWER_DATASET_DIRECTORY, FLOWER_DATASET_ARCHIVE_FILENAME), FLOWER_DATASET_DIRECTORY)
    

### Create the dataset and dataloader

In [4]:
classes = []
for file in os.listdir(os.path.join(FLOWER_DATASET_DIRECTORY, 'flower_photos')):
    class_dir = os.path.join(FLOWER_DATASET_DIRECTORY, 'flower_photos', file)
    if os.path.isdir(class_dir):
        new_class = []
        for image in os.listdir(class_dir):
            new_class.append(os.path.join(class_dir, image))
        classes.append(new_class)

dataset = []
for i, cl in enumerate(classes):
    for sample in enumerate(cl):
        dataset.append({ 'class': i, 'path': sample })

train_paths, validate_paths, test_paths = random_split(dataset, [TRAIN_DATASET_RATIO, VAL_DATASET_RATIO, TEST_DATASET_RATIO])

In [5]:
class FlowerClassificationDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        image = read_image(self.samples[idx]['path'][1])
        

        image = image.float() / 255

        if self.transform:
            image = self.transform(image)

        return image, self.samples[idx]['class']


In [6]:
transforms = v2.Resize(size=(224, 224))

train_dataset = FlowerClassificationDataset(train_paths, transforms)
val_dataset = FlowerClassificationDataset(validate_paths, transforms)
test_dataset = FlowerClassificationDataset(test_paths, transforms)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=5)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=5)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=5)

### Load the model

In [7]:
class MuseumCC0Module(LightningModule):
    def __init__(self):
        super().__init__()
        self.loss_module = torch.nn.CrossEntropyLoss()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None)
        self.out_features = self.model.fc.in_features
        self.model.fc = torch.nn.Identity()

    def forward(self, imgs):
        return self.model(imgs)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.1)   


def get_newest_model():
    logs_paths = os.path.join(CHECKPOINTS_DIRECTORY, 'lightning_logs')
    newest_version = -1
    checkpoint_path = None
    for dir in os.listdir(logs_paths):
        dir_path = os.path.join(logs_paths, dir)
        if os.path.isdir(dir_path) and 'version_' in dir:
            num = int(dir.replace('version_', ''))
            if newest_version > num:
                continue
            
            listing = os.listdir(dir_path)
            if 'checkpoints' in listing:
                check_dir = os.path.join(dir_path, 'checkpoints')
                for filename in os.listdir(check_dir):
                    if '.ckpt' in filename:
                        checkpoint_path = os.path.join(check_dir, filename)
                        newest_version = num

    print(f'Using checkpoint version {newest_version}!') 
    return checkpoint_path

if USE_RANDOM_MODEL:
    base_model = MuseumCC0Module()
else:
    base_model = MuseumCC0Module.load_from_checkpoint(get_newest_model(), strict=False)
in_features = base_model.out_features

Using checkpoint version 46!


Using cache found in /home/macron/.cache/torch/hub/pytorch_vision_v0.10.0
/home/macron/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['model.fc.weight', 'model.fc.bias']


In [8]:

class FlowerClassificationModule(LightningModule):
    def __init__(self, model, in_features):
        super().__init__()
        self.model = model

        for param in self.model.parameters():
            param.requires_grad = False
        
        self.out = torch.nn.Sequential(
            torch.nn.Linear(in_features, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(64, 5),
        )
        self.loss_module = torch.nn.CrossEntropyLoss()

    def forward(self, imgs):
        x = self.model(imgs)
        return self.out(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.out.parameters(), lr=LEARNING_RATE)   

    def training_step(self, batch, batch_idx):
        images, labels = batch

        x = self.model(images)
        preds = self.out(x)
        
        loss = self.loss_module(preds, labels)

        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log("train_acc", acc, on_step=True, on_epoch=True)
        self.log("train_loss", loss)
        
        return loss 

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        
        x = self.model(images)
        preds = self.out(x)

        loss = self.loss_module(preds, labels)

        acc = (labels == preds.argmax(dim=-1)).float().mean()

        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        images, labels = batch
        
        x = self.model(images)
        preds = self.out(x)

        loss = self.loss_module(preds, labels)

        acc = (labels == preds.argmax(dim=-1)).float().mean()

        self.log("test_acc", acc, on_epoch=True, prog_bar=True)
        self.log("test_loss", loss, on_epoch=True, prog_bar=True)

model = FlowerClassificationModule(base_model, in_features)

### Train model

In [9]:
trainer = Trainer(default_root_dir=FLOWER_CHECKPOINTS_DIRECTORY, max_epochs=EPOCHS)

trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | MuseumCC0Module  | 23.5 M
1 | out         | Sequential       | 1.2 M 
2 | loss_module | CrossEntropyLoss | 0     
-------------------------------------------------
1.2 M     Trainable params
23.5 M    Non-trainable params
24.7 M    Total params
98.920    Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


### Evaluate the model
The ImageNet-trained has managed to get loss of 0.3605 and accuracy of 88.71%. 

In [10]:
trainer.test(dataloaders=test_dataloader)

/home/macron/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at flowers-checkpoints/lightning_logs/version_36/checkpoints/epoch=9-step=810.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at flowers-checkpoints/lightning_logs/version_36/checkpoints/epoch=9-step=810.ckpt


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.3351498544216156     │
│         test_loss         │    1.5038419961929321     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.3351498544216156, 'test_loss': 1.5038419961929321}]